In [1]:
# import libraries 

from bs4 import BeautifulSoup
import requests
from time import sleep
import datetime
import lxml
import smtplib
from faker import Faker
import pandas as pd

In [14]:
webdriver_path = '/path/to/chromedriver'
driver = webdriver.Chrome(executable_path=webdriver_path)
driver.get('https://www.amazon.com/')

# Wait for user to interact with the browser or perform some actions
# For example, you can instruct the user to navigate to a product page on Amazon manually

# Get the current URL of the browser
url = driver.current_url

# Print the URL to the console or store it in your database
print("Current URL:", url)

# Close the browser
driver.quit()

NameError: name 'webdriver' is not defined

In [4]:
# Connect to Website and pull in data
reviewlist = []
def get_soup_retry(url):
    fake = Faker()
    uag_random = fake.user_agent()

    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        page = requests.get(url, headers=header)
        assert page.status_code == 200
        soup = BeautifulSoup(page.content, 'lxml')
        if 'captcha' in str(soup):
            uag_random = fake.user_agent()
            print(f'\rBot has been detected... retrying ... use new identity: {uag_random} ', end='', flush=True)
            continue
        else:
            print('Bot bypassed')
            return soup

def extractReviews(url):
    soup = get_soup_retry(url)

    headers = {'authority': 'www.amazon.com',
                'pragma': 'no-cache',
                'cache-control': 'no-cache',
                'dnt': '1',
                'upgrade-insecure-requests': '1',
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1",
                'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'sec-fetch-site': 'none',
                'sec-fetch-mode': 'navigate',
                'sec-fetch-dest': 'document',
                'accept-language': 'en-US;q=0.9,en;q=0.8',}
    reviews = soup.findAll('div', {'data-hook':"review"})
    title = ' '
    r_title= ' '
    rating = ' '
    body = ' '
    
    # print(reviews)
    for item in reviews:
            try:
                title = soup.title.text.replace("Amazon.com: Customer reviews: ", "").strip()

            except AttributeError:
                title = 'N'
            try:
                r_title = item.find('a', {'data-hook':"review-title"}).text.strip().split('\n')
                r_title = r_title[1]
            except AttributeError:
                r_title = 'N'
            try:
                rating = item.find('i', {'data-hook': 'review-star-rating'}).text.strip("('')")
                rating= rating[0:3]
            except AttributeError:
                rating = 'N'
            try:
                body = item.find('span', {'data-hook': 'review-body'}).text.strip() 
            except AttributeError:
                body = 'N'
            review = {
                "Product Name" : title,
                "Review Header": r_title,
                "Rating": rating,
                "Review Body" : body
            }
            reviewlist.append(review)

def main():
    url = 'https://www.amazon.com/Kokuyo-Campus-Notebook-Semi-B5-NO-3CATNX5/dp/B002C4KL88/?_encoding=UTF8&pd_rd_w=RfW2q&content-id=amzn1.sym.5f7e0a27-49c0-47d3-80b2-fd9271d863ca%3Aamzn1.symc.e5c80209-769f-4ade-a325-2eaec14b8e0e&pf_rd_p=5f7e0a27-49c0-47d3-80b2-fd9271d863ca&pf_rd_r=4X5H702HEFJEY5RY2A9B&pd_rd_wg=LxQa0&pd_rd_r=43f615dc-3e3a-403b-89d2-ac483baf1929&ref_=pd_gw_ci_mcx_mr_hp_atf_m'
    reviewUrl = url.replace("dp", "product-reviews") 
    extractReviews(reviewUrl)
    if reviewlist ==[]:
        reviewUrl = url.replace("dp", "product-reviews") 
        extractReviews(reviewUrl)
    df = pd.DataFrame(reviewlist)
    df.to_excel('output.xlsx', index=False)

main()


Bot has been detected... retrying ... use new identity: Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 5.0; Trident/4.0) ko) Chrome/61.0.811.0 Safari/536.0 .0.878.0 Mobile/65M595 Safari/532.0 Bot bypassed


In [5]:
# import excel as a data frame and turning it into a string
df = pd.read_excel('output.xlsx')
reviews = df['Review Body']
reviews_string = ' '.join(map(str,reviews))
#reviews_string

In [26]:
# using google T5 abstractive summarization
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

# initialize the tokenizer model
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

# input data into the code below to tokenize it:
inputs = tokenizer.encode("summarize: " + reviews_string, return_tensors='pt', max_length=512, padding='max_length', 
                          truncation=True)

# generate the summary by using the model.generate function on T5
summary_ids = model.generate(inputs, max_length=400, min_length=100, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:")
print(summary)

Generated Summary:
for $10, you get five notebooks in purple, blue, green, golden yellow, and pink. each notebook contains 30 pages (=60 pages front and back) the quality of the paper is amazing, and there's a little bit of show-through. the cover helps tremendously in keeping the corners from folding in my backpack. it's hard to tell if the notebooks are damaged, but it's a great value for the money.


In [76]:
# using nltk to summarize extractive
'''    
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize, sent_tokenize


    # tokenize the text:
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(reviews_string)

    # create a frequency table to keep a score of each word:
    freqTable = dict()
    for word in words:
        word = word.lower()
        if word in stopWords:
            continue
        if word in freqTable:
            freqTable[word] += 1
        else:
            freqTable[word] = 1

    # create a dictionary to keep the score of each sentence:
    sentences = sent_tokenize(reviews_string)
    sentenceValue = dict()

    for sentence in sentences:
            for word, freq in freqTable.items():
                if word in sentence.lower():
                    if word in sentence.lower():
                        if sentence in sentenceValue:
                            sentenceValue[sentence] += freq
                        else:
                            sentenceValue[sentence] = freq
    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    # define the average value from the original text as such:
    average = int(sumValues / len(sentenceValue))

    #store the sentences into our summary:
    summary = ''
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.7 * average)):
            summary += " " + sentence
    print(summary)
    '''

 For $10, you get five notebooks in purple, blue, green, golden yellow, and pink, each containing 30 pages (=60 pages front and back). One might say they're expensive, given that you can easily buy a composition book for 50 cents during back-to-school sales, but I just adore how slim these notebooks are, and the quality of the paper is amazing. Except for the very first and last pages, the notebooks open to a relatively flat position, or at least flatter than a composition book can open up.I was initially hesitant to buy these books because of how little pages each notebook has, but I jumped for it when I realized I have trouble finishing a standard composition book. So maybe I'd recommend buying other items to be shipped with your notebooks?Since using this textbook in class, I've found that, for me, a chapter's worth of notes in my economics class takes up only one page (front and back). I'm thinking of buying the 6mm notebooks simply because I'd like the extra lines per page, but th

In [ ]:
# tracks historical price and suggests a month or day to buy the product at its lowest


In [ ]:
# If uou want to try sending yourself an email (just for fun) when a price hits below a certain level you can try it
# out with this script

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('AlexTheAnalyst95@gmail.com','xxxxxxxxxxxxxx')
    
    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = "Alex, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data+analyst+tshirt&qid=1626655184&sr=8-3"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'AlexTheAnalyst95@gmail.com',
        msg
     
    )